In [1]:
#export
"""
This module is for communication with hardware. This is exposed automatically with::

   from k1lib.imports import *
   kcom.Gsm # exposed
"""
import k1lib, base64, io, os, time; import k1lib.cli as cli; import numpy as np; from collections import deque
serial = k1lib.dep("serial", "pyserial", "https://pyserial.readthedocs.io")
__all__ = ["Gsm", "Host"]

In [13]:
#export
class Gsm:
    def __init__(self, device="/dev/ttyUSB0"):
        """Communicates with a GSM module using AT codes via a serial device.
Example::

    g = kcom.Gsm("/dev/ttyUSB0")
    g # display in cell, will show basic info
    
    g.sendSms("+1215123456", "some message")
    s.readSms()
"""
        self.device = device
        self.conn = serial.Serial(device)
    def sendRaw(self, data:bytes):
        """Sends data to the module and read response until 'OK\\n' is received"""
        if isinstance(data, str): data = data.encode()
        if not isinstance(data, bytes): raise Exception("Data has to be bytes")
        self.conn.write(data + b"\n"); res = b""
        while True:
            time.sleep(0.003); res += self.conn.read_all().replace(b"\r", b"")
            if res[-3:] == b"OK\n": return res
    def send(self, data:bytes):
        """Similar to :meth:`sendRaw`, but this time clean up control signals like 'OK\\n'"""
        res = self.sendRaw(data); splits = deque(res.split(b"\n")); splits.popleft()
        while len(splits) and splits[-1] == b"": splits.pop()
        if len(splits) and splits[-1] == b"OK": splits.pop()
        while len(splits) and splits[-1] == b"": splits.pop()
        return b"\n".join(splits)
    def sendSms(self, number:str, data:bytes):
        """Sends text message to some number."""
        if isinstance(data, str): data = data.encode()
        if not isinstance(data, bytes): raise Exception("Data has to be bytes")
        return self.send(f'AT+CMGS="{number}"\n'.encode() + data + b"\x1a")
    def readSms(self, mode=1):
        """Reads text messages.

:param mode: 0 for all, 1 for unread, 2 for read"""
        self.send(b"AT+CMGF=1\n") # sets text mode
        if mode == 0: return self.send(b'AT+CMGL="ALL"')
        elif mode == 1: return self.send(b'AT+CMGL="REC UNREAD"')
        elif mode == 2: return self.send(b'AT+CMGL="REC READ"')
    def close(self): self.conn.close()
    def __repr__(self):
        return [
            ["International Mobile Subscriber Identity", self.send(b"AT+CIMI")],
            ["Ready?", self.send(b"AT+CPIN?")],
            ["Cellular network registration", self.send(b"AT+CREG?")],
            ["Signal strength & bit error rate", self.send(b"AT+CSQ")],
            ["General Packet Radio Service (aka data)", self.send(b"AT+CGATT?")],
            ["Packet Data Protocol context", self.send(b"AT+CGACT?").replace(b"\n", b" - ")]] | cli.apply(cli.op().decode(), 1) | cli.pretty() | cli.join("\n")

In [3]:
#export
import shlex
_host_baseFn = [None]; _host_autoInc = k1lib.AutoIncrement()
def _host_getTmpDir():
    if _host_baseFn[0] is None: _host_baseFn[0] = b"" | cli.file(); os.remove(_host_baseFn[0])
    return _host_baseFn[0] + f"_{int(time.time())}_{_host_autoInc()}_{os.getpid()}"
def _host_stripOut(out:"list[bytes]", preds):
    out = deque(out)
    while len(out) > 0: # removing annoying log messages from ssh and bash
        broken = False
        for pred in preds:
            if pred(out[0]): out.popleft(); broken = True; break
        if not broken: break
    return list(out)
class Host:
    def __init__(self, user=None, host=None, container=None, verbose=False, password=None, stripPredicates=[]):
        """Represents another computer that you might want to execute commands in.
Examples::

    h = kcom.Host("username", "hostname")
    h.execPy("print('something')", fns=["~/.bashrc"], rmFn=False) # returns [List[bytes], List[bytes], List[bytes]]

There are several available modes::

    h = kcom.Host(user="username", host="hostname")     # Mode 1: executes in ssh host
    h = kcom.Host(container="nginx-1")                  # Mode 2: executes in container on localhost
    h = kcom.Host(host="hostname", container="nginx-1") # Mode 3: executes in container on remote ssh host

Mode 1 and 2 are relatively straightforward and airtight. Mode 3 is a little buggy
in stdout and stderr. Once you have constructed them, you can execute random
python/bash scripts in the remote host::

    h.execPy("import os; print(os.environ)")

Main value of this class is that it can execute random scripts in a convenient
manner. I've found myself needing this way more often than expected.

:param user: username of the ssh host
:param host: host name of the ssh host
:param container: container name. If specified will run commands inside that container
:param verbose: if True, will print out detailed commands that're executed and their results
:param password: if True, will try to login to ssh host using a password instead of the default key file
:param stripPredicates: list of predicates that if match, will delete those first few lines in stdout
    and stderr. This is to strip away annoying boilerplate messages that I couldn't quite get rid of myself
"""
        self.user = user; self.host = host; self.container = container; self._connStatus = None; self.verbose = verbose; self.password = password; self.stripPredicates = [cli.op.solidify(f) for f in [lambda x: x.startswith(b"Connection to "), *stripPredicates]]
        self._userHost = None if host is None else (host if user is None else f"{user}@{host}"); self.pwPrefix = f"sshpass -p {shlex.quote(password)} " if password else ""
    def _connCheck(self):
        if len(self._exec1("pwd")) == 0: raise Exception(f"Can't connect. Please check your ssh/docker permissions")
    def _exec1(self, c): # execute normal commands, no input expected, just output
        if self.verbose: print(f"executing command: {repr(c)}")
        if self.container is None: # ssh only
            c = f"{self.pwPrefix}ssh {self._userHost} {shlex.quote(c)}"
            if self.verbose: print(f"-----command: {repr(c)}")
            res = None | cli.cmd(c, mode=0, text=False) | cli.deref()
            if self.verbose: print(f"-----res: {repr(res)}")
            return res
        elif self.host is None: # container only
            c = f"docker exec -i {self.container} sh -c {shlex.quote(c)}"
            if self.verbose: print(f"-----command: {repr(c)}")
            res = None | cli.cmd(c, mode=0, text=False) | cli.deref()
            if self.verbose: print(f"-----res: {repr(res)}")
            return res
        else: # ssh and container
            d = f"docker exec -i {self.container} sh -c {shlex.quote(c)}"; e = f"bash -ic {shlex.quote(d)}"
            # e = shlex.quote(f"{{ bash -ic {shlex.quote(d)} }} 2>/dev/null")
            c = f"{self.pwPrefix}ssh {self._userHost} -tt {shlex.quote(e)}"
            if self.verbose: print(f"-----command: {repr(c)}")
            res = None | cli.cmd(c, mode=0, text=False) | cli.apply(_host_stripOut, preds=self.stripPredicates) | cli.deref()
            if self.verbose: print(f"-----res: {repr(res)}")
            return res
    def _exec2(self, fnLocal, fnRemote): # send file to remote, no output expected, just input
        if self.verbose: print(f"transferring file {repr(fnLocal)} to {repr(fnRemote)}")
        if self.container is None: # ssh only
            d = f"cat > {shlex.quote(fnRemote)}"; c = f"({self.pwPrefix}ssh {self._userHost} -T {shlex.quote(d)}) < {shlex.quote(fnLocal)}"
            if self.verbose: print(f"-----command: {repr(c)}")
            res = None | cli.cmd(c, mode=0, text=False) | cli.deref()
            if self.verbose: print(f"-----res: {repr(res)}")
            return res
        elif self.host is None: # container only
            d = f'cat > {shlex.quote(fnRemote)}'; c = f"(docker exec -i {self.container} sh -c {shlex.quote(d)}) < {shlex.quote(fnLocal)}"
            if self.verbose: print(f"-----command: {repr(c)}")
            res = None | cli.cmd(c, mode=0, text=False) | cli.deref()
            if self.verbose: print(f"-----res: {repr(res)}")
            return res
        else: # ssh and container. TODO: both stdout and stderr goes to stdout, and stderr has some filler material from logging in. Need to fix, but am lazy
            c = f"cat > {shlex.quote(fnRemote)}"; d = f"docker exec -i {self.container} sh -c {shlex.quote(c)}"; e = f"bash -ic {shlex.quote(d)}"
            c = f"({self.pwPrefix}ssh {self._userHost} -t {shlex.quote(e)}) < {shlex.quote(fnLocal)}"
            if self.verbose: print(f"-----command: {repr(c)}")
            res = None | cli.cmd(c, mode=0, text=False) | cli.deref()
            if self.verbose: print(f"-----res: {repr(res)}")
            return res
    def _exec(self, c, fns, rmFn, executable):
        _userHost = self._userHost; tmpDir = _host_getTmpDir()
        self._connCheck(); None | cli.cmd(f"mkdir -p {shlex.quote(tmpDir)}") | cli.deref(); self._exec1(f"mkdir -p {shlex.quote(tmpDir)}")
        c | cli.file(f"{tmpDir}/script"); self._exec2(f"{tmpDir}/script", f"{tmpDir}/script")
        out, err = self._exec1(f"{shlex.quote(executable)} {shlex.quote(tmpDir + '/script')}")
        if fns is not None:
            files = fns | cli.apply(lambda fn: self._exec1(f"cat {shlex.quote(fn)}") | cli.item()) | cli.deref()
            if rmFn: fns | cli.apply(lambda fn: self._exec1(f"rm {shlex.quote(fn)}") | cli.item()) | cli.deref()
        else: files = []
        None | cli.cmd(f"rm -rf {shlex.quote(tmpDir)}") | cli.deref(); self._exec1(f"rm -rf {shlex.quote(tmpDir)}"); return [out, err, *files]
    def execPy(self, c:str, fns=None, rmFn=False, pyExec=None):
        """Executes some python code.
Examples::

If .fns is not specified, then will return (List[bytes], List[bytes]) containing (stdout, stderr).

If .fns is specified, then will return (List[bytes], List[bytes], List[bytes], ...) containing
(stdout, stderr, fn1, fn2, ...). Each file is a List[bytes], with endline byte at the end of each bytes chunk

:param c: Python commands
:param fns: file names to retrieve. Don't use paths that can expand, like "~/.bashrc" or
    "$HOME/.bashrc", they won't be expanded
:param rmFn: if True, removes the files after running and exiting, else don't remove the
    files. Some scripts auto generates files that should be removed, but others don't
:param pyExec: py executable used to run the generated python file, like "/usr/bin/python". If
    not specified will try to auto detect what python binaries are available
"""
        if pyExec is None:
            ans = self._exec1("which python")
            if len(ans[0]) == 0: raise Exception("No python binary on PATH found. Please manually specify the location of the Python binary by setting .pyExec")
            pyExec = ans[0][-1].decode().strip("\r\n")
        return self._exec(c, fns, rmFn, pyExec)
    def execSh(self, c:str, fns=None, rmFn=False, shExec="/bin/bash"):
        """Executes some bash code. Pretty much the same
as :meth:`execPy`, but for bash. See that method's docs."""
        return self._exec(c, fns, rmFn, shExec)
    def __repr__(self):
        if self._connStatus is None: self._connStatus = len(self._exec1("pwd")[0]) > 0
        u = f"user='{self.user}' " if self.user else ""; h = f"host='{self.host}' " if self.host else ""
        c = f"container='{self.container}' " if self.container else ""
        return f"<Host {u}{h}{c}connected={self._connStatus}>"

In [4]:
%%time
h = Host("kelvin", "mint-3.l"); h.__repr__(); assert h._connStatus
ans = h.execSh("ls ~/repos", fns=["/home/kelvin/.bashrc"], rmFn=False)
assert len(ans[0]) > 5; assert len(ans[1]) == 0; assert len(ans[2]) > 20
ans = h.execPy("import warnings; print('something'); warnings.warn('some warnings')", fns=["/home/kelvin/.bashrc"], rmFn=False)
assert len(ans[0]) == 1; assert ans[0][0] == b"something\n"; assert len(ans[1]) == 2; assert len(ans[2]) > 20

CPU times: user 132 ms, sys: 196 ms, total: 328 ms
Wall time: 17.3 s


In [5]:
%%time
h = Host(container="server-nginx-1"); h.__repr__(); assert h._connStatus
ans = h.execSh("ls /etc/nginx", fns=["/etc/nginx/nginx.conf"], rmFn=False)
assert len(ans[0]) > 14; assert len(ans[1]) == 0; assert len(ans[2]) > 30
h = Host(container="server-grafana-py-1"); h.__repr__(); assert h._connStatus
ans = h.execPy("import warnings; print('something'); warnings.warn('some warnings')", pyExec="/root/miniconda3/bin/python", fns=[], rmFn=False)
assert len(ans[0]) == 1; assert ans[0][0] == b"something\n"; assert len(ans[1]) == 2

CPU times: user 13.7 ms, sys: 133 ms, total: 147 ms
Wall time: 827 ms


In [6]:
%%time
h = Host(user="kelvin", host="ny.l", container="front", verbose=False); h.__repr__(); assert h._connStatus
ans = h.execSh("ls /etc/haproxy", fns=["/etc/haproxy/haproxy.cfg"], rmFn=False)
assert len(ans[0]) == 2; assert len(ans[1]) == 0; assert len(ans[2]) > 20

CPU times: user 93.6 ms, sys: 83 ms, total: 177 ms
Wall time: 11 s


rmFn tests:

In [7]:
%%time
h = Host(container="server-nginx-1"); h.__repr__(); assert h._connStatus, 0
assert len(h.execSh("echo 'inside facb' > fabc")[0]) == 0, 1
assert h.execSh("ls | grep fabc") == [[b'fabc\n'], []], 2
assert h.execSh("pwd", fns=["fabc"]) == [[b'/code\n'], [], [b'inside facb\n']], 3
assert h.execSh("ls | grep fabc") == [[b'fabc\n'], []], 4
assert h.execSh("pwd", fns=["fabc"], rmFn=True) == [[b'/code\n'], [], [b'inside facb\n']], 5
assert h.execSh("ls | grep fabc") == [[], []], 6
assert h.execSh("pwd", fns=["fabc"], rmFn=True) == [[b'/code\n'], [], []], 7

CPU times: user 34.2 ms, sys: 475 ms, total: 509 ms
Wall time: 2.68 s


In [8]:
%%time
h = Host(user="kelvin", host="mint-2.l", container="server-nginx-1"); h.__repr__(); assert h._connStatus, 0
assert len(h.execSh("echo 'inside facb' > fabc")[0]) == 0, 1
assert h.execSh("ls | grep fabc") == [[b'fabc\r\n'], []], 2
assert h.execSh("pwd", fns=["fabc"]) == [[b'/code\r\n'], [], [b'inside facb\r\n']], 3
assert h.execSh("ls | grep fabc") == [[b'fabc\r\n'], []], 4
assert h.execSh("pwd", fns=["fabc"], rmFn=True) == [[b'/code\r\n'], [], [b'inside facb\r\n']], 5
assert h.execSh("ls | grep fabc") == [[], []], 6
assert h.execSh("pwd", fns=["fabc"], rmFn=True) == [[b'/code\r\n'], [], [b'cat: fabc: No such file or directory\r\n']], 7 # TODO: this last statement might be considered out of spec, but too lazy to fix at this point

CPU times: user 921 ms, sys: 596 ms, total: 1.52 s
Wall time: 1min 32s


In [12]:
!../export.py kcom --upload=True

2024-03-05 23:13:25,787	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-03-05 23:13:25,796	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15514   0   61%   
9963    1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.5.2
Uninstalling k1lib-1.5.2:
  Successfully uninstalled k1lib-1.5.2
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdi

In [15]:
!../export.py kcom

2024-03-08 06:36:19,051	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-03-08 06:36:19,060	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15682   0   61%   
10043   1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.6
Uninstalling k1lib-1.6:
  Successfully uninstalled k1lib-1.6
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg

In [5]:
!../export.py kcom --bootstrap=True

2024-02-04 18:25:04,535	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2024-02-04 18:25:04,549	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- bootstrapping
Current dir: /home/kelvin/repos/labs/k1lib, /home/kelvin/repos/labs/k1lib/k1lib/../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.5.2
Uninstalling k1lib-1.5.2:
  Successfully uninstalled k1lib-1.5.2
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
r